In [1]:
%load_ext autoreload
%autoreload

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OneHotEncoder
import re
from keras.layers import Input, Dense
from keras.models import Model, load_model
from keras.callbacks import EarlyStopping, TensorBoard
from keras import metrics
import json
from helper_functions import import_data

#plt.style.use('seaborn')
plt.style.use('ggplot')

Using TensorFlow backend.


In [54]:
y_val_length = y_vals.shape[0]


ohe = OneHotEncoder(categories='auto')
ohe.fit(y_vals.reshape([-1, 1]))
ohe.transform(y_vals.reshape([-1, 1]))

<4683x2 sparse matrix of type '<class 'numpy.float64'>'
	with 4683 stored elements in Compressed Sparse Row format>

In [72]:

new_y = np.zeros([y_vals.shape[0], 2])
new_y[:, 0] = y_vals
new_y[:, 1] = new_y[:, 1] + 1
new_y[:, 1] = new_y[:, 1] - new_y[:, 0]
np.set_printoptions(threshold=30)
new_y.shape

(4683, 2)

(536, 2)

In [34]:
dense_matrix, y_vals = import_data(test_json_path)


In [3]:
test_json_path = 'jsons/part-00001-3a3b13a1-b857-4d1c-965f-ce7bf860f3bc-c000.json'


In [9]:
dense_matrix, y_vals = import_data(test_json_path)
X_test = dense_matrix[0:1000, :]
X_train = dense_matrix[1000:, :]

y_test = y_vals[0:1000]
y_train = y_vals[1000:]

(4683, 15133)

In [15]:
def autoencoder_model(X_train):
    '''
    defines autoencoder model
    input: X_train (2D np array)
    output: autoencoder (compiled autoencoder model)
    '''
    # this is our input placeholder
    input_img = Input(shape=(X_train.shape[1],))

    # first encoding layer
    encoded1 = Dense(units = 256, activation = 'relu', name='layer1_256')(input_img)

    # second encoding layer
    # note that each layer is multiplied by the layer before
    encoded2 = Dense(units = 64, activation='relu', name='layer2_64')(encoded1)

    # first decoding layer
    decoded1 = Dense(units = 256, activation='relu', name='layer3_256')(encoded2)

    # second decoding layer - this produces the output
    decoded2 = Dense(units = X_train.shape[1], activation='sigmoid', name='layer4_output')(decoded1)

    # this model maps an input to its reconstruction
    autoencoder = Model(input_img, decoded2)

    # compile model
    autoencoder.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics=['mse'])

    return autoencoder

In [18]:
def batch_generator(X, batch_size, y=[]):
    number_of_batches = samples_per_epoch/batch_size
    counter=0
    shuffle_index = np.arange(np.shape(X)[0])
    np.random.shuffle(shuffle_index)
    
    while 1:
        index_batch = shuffle_index[batch_size*counter:batch_size*(counter+1)]
        X_batch = X[index_batch]
        counter += 1
        
        if y == []:
            yield X_batch, X_batch
        else:
            yield X_batch, y[index_batch]
        
        
        if (counter > number_of_batches):
            np.random.shuffle(shuffle_index)
            counter=0

In [21]:
autoencoder_model_created = False
model_path = 'models/basic_autoencoder1.h5'

if not autoencoder_model_created:
    model = autoencoder_model(X_train)

    batch_size = 1000
    nb_epoch = 10
    samples_per_epoch = 10

    # instantiate callbacks
    tensorboard = TensorBoard(log_dir='./autoencoder_logs', histogram_freq=2, batch_size=batch_size, write_graph=True, write_grads=True, write_images=True)
    earlystopping = EarlyStopping(monitor='val_loss', patience=2)

    # try different number of epochs - 10 gives good performanace 
    """model.fit(X_train, X_train, epochs=10, batch_size=batch_size, verbose=1,
              validation_split=0.1, callbacks = [earlystopping, tensorboard])""" # cross val to estimate test error


    model.fit_generator(generator=batch_generator(X_train, batch_size),
                        epochs=nb_epoch,
                        steps_per_epoch=samples_per_epoch)


    scores = model.evaluate(X_test, X_test)
    print('Test mse = {}'.format(scores[0]))

    X_test_decoded = model.predict(X_test)
    
    model.save(model_path)

else:
    model = load_model(model_path)
    scores = model.evaluate(X_test, X_test)
    print('Test mse = {}'.format(scores[0]))


Epoch 1/10
10/10 [==============================] - 1s 146ms/step - loss: 0.2599 - mean_squared_error: 0.2599
Epoch 2/10
10/10 [==============================] - 1s 93ms/step - loss: 0.1173 - mean_squared_error: 0.1173
Epoch 3/10
10/10 [==============================] - 1s 92ms/step - loss: 0.0403 - mean_squared_error: 0.0403
Epoch 4/10
10/10 [==============================] - 1s 95ms/step - loss: 0.0390 - mean_squared_error: 0.0390
Epoch 5/10
10/10 [==============================] - 1s 95ms/step - loss: 0.0413 - mean_squared_error: 0.0413
Epoch 6/10
10/10 [==============================] - 1s 95ms/step - loss: 0.0391 - mean_squared_error: 0.0391
Epoch 7/10
10/10 [==============================] - 1s 97ms/step - loss: 0.0333 - mean_squared_error: 0.0333
Epoch 8/10
10/10 [==============================] - 1s 96ms/step - loss: 0.0320 - mean_squared_error: 0.0320
Epoch 9/10
10/10 [==============================] - 1s 94ms/step - loss: 0.0379 - mean_squared_error: 0.0379
Epoch 10/10
1000/1

In [10]:
for i in range(4):
    print(model.layers[i].name)

input_2
layer1_256
layer2_64
layer3_256


In [ ]:
for i in range(4):
    print(model.layers[i].name)
    model.layers[i].trainable = False

In [22]:
ll = model.layers[3].output
ll = Dense(units = 64, activation='relu', name='layer4_256')(ll)
ll = Dense(2,activation="hard_sigmoid", name='positive_classification')(ll)
new_model = Model(inputs=model.input, outputs=ll)

new_model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics=[metrics.categorical_accuracy])

In [23]:
batch_size = 500
nb_epoch = 15
samples_per_epoch = 10
model_path = 'models/basic_autoencoder1.h5'

# instantiate callbacks
tensorboard = TensorBoard(log_dir='./autoencoder_logs', histogram_freq=2, batch_size=batch_size, write_graph=True, write_grads=True, write_images=True)
earlystopping = EarlyStopping(monitor='val_loss', patience=2)

# try different number of epochs - 10 gives good performanace 
"""model.fit(X_train, X_train, epochs=10, batch_size=batch_size, verbose=1,
          validation_split=0.1, callbacks = [earlystopping, tensorboard])""" # cross val to estimate test error


new_model.fit_generator(generator=batch_generator(X_train, batch_size, y_train),
                    epochs=nb_epoch,
                    steps_per_epoch=samples_per_epoch)


scores = new_model.evaluate(X_test, y_test)
print('Test accuracy = {}'.format(scores[1]))

X_test_decoded = new_model.predict(X_test)

new_model.save(model_path)

Epoch 1/15


/home/tyler/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if sys.path[0] == '':


ValueError: Error when checking target: expected positive_classification to have shape (2,) but got array with shape (1,)

In [17]:
"""enc = OneHotEncoder()
enc.fit(y_test.reshape(-1,1))
enc.transform(y_test.reshape(-1,1)).toarray()
enc.transform(y_train.reshape(-1,1)).toarray()"""

/home/tyler/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [1., 0.],
       [1., 0.],
       [0., 1.]])